### CCF API UseCase Documentation

[Click here](https://github.com/hubmapconsortium/ccf-ui/blob/main/ccf-api-usage.ipynb) to view the installation and general documentation for ccf-api python module.

In [3]:
import ccf_openapi_client
from ccf_openapi_client.api import default_api
from ccf_openapi_client.model.aggregate_count import AggregateCount
from ccf_openapi_client.model.database_status import DatabaseStatus
from ccf_openapi_client.model.error_message import ErrorMessage
from ccf_openapi_client.model.flat_spatial_placement import FlatSpatialPlacement
from ccf_openapi_client.model.get_spatial_placement_request import GetSpatialPlacementRequest
from ccf_openapi_client.model.min_max import MinMax
from ccf_openapi_client.model.ontology_tree import OntologyTree
from ccf_openapi_client.model.sparql_query_request import SparqlQueryRequest
from ccf_openapi_client.model.spatial_entity import SpatialEntity
from ccf_openapi_client.model.spatial_scene_node import SpatialSceneNode
from ccf_openapi_client.model.spatial_search import SpatialSearch
from ccf_openapi_client.model.tissue_block import TissueBlock
from pprint import pprint

In [4]:
import pandas as pd
import csv

In [5]:
# Configuring the API Client
configuration = ccf_openapi_client.Configuration(
    host = "https://ccf-api.hubmapconsortium.org/v1"
)
api_client = ccf_openapi_client.ApiClient(configuration)

api_instance = default_api.DefaultApi(api_client)

##### Check Database Status

In [6]:
db_ready = False
result = None
while not db_ready:
    result = api_instance.db_status()
    if result['status'] == 'Ready':
        db_ready = True
    else:
        print('Database not ready yet! Retrying...', result)
        time.sleep(2)
print('Database ready!\n', result)

Database ready!
 {'checkback': 3600000,
 'load_time': 12104,
 'message': 'Database successfully loaded',
 'status': 'Ready'}


The `scene` api 

In [105]:
sex = "both"
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0002113",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",]
sceneResult = None
try:
    sceneResult = api_instance.scene(sex=sex, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms)
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)
sceneResult

[{'@id': 'http://purl.org/ccf/latest/ccf.owl#VHMSkinV1.1',
  '@type': 'SpatialSceneNode',
  'color': [255.0, 255.0, 255.0, 255.0],
  'lighting': 'pbr',
  'opacity': 0.5,
  'reference_organ': 'http://purl.org/ccf/latest/ccf.owl#VHMSkinV1.1',
  'representation_of': 'http://purl.obolibrary.org/obo/UBERON_0002097',
  'scenegraph': 'https://ccf-ontology.hubmapconsortium.org/objects/v1.2/VH_M_Skin.glb',
  'scenegraph_node': 'VH_M_skin',
  'tooltip': 'Skin',
  'transform_matrix': [1.0,
                       0.0,
                       0.0,
                       0.0,
                       0.0,
                       1.0,
                       0.0,
                       0.0,
                       0.0,
                       0.0,
                       1.0,
                       0.0,
                       -0.4,
                       0.0,
                       0.0,
                       1.0],
  'unpickable': True,
  'zoom_based_opacity': False},
 {'@id': 'http://purl.org/ccf/latest/ccf

In [8]:
sceneEntityASDict = {}
for scene in sceneResult:
    if 'entity_id' in scene:
        if scene['entity_id'] not in sceneEntityASDict:
            sceneEntityASDict[scene['entity_id']] = []
        sceneEntityASDict[scene['entity_id']].extend(scene['ccf_annotations'])

In [9]:
query = '''PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ccf: <http://purl.org/ccf/>

SELECT DISTINCT (STR(?asLabel) as ?as_label) (STR(?qlabel) as ?cell_label) ?as_iri ?cell_iri WHERE {
  ?cell_iri ccf:ccf_located_in ?as_iri .
  ?cell_iri rdfs:label ?qlabel .
  ?as_iri rdfs:label ?asLabel .

  FILTER (?as_iri in (%s))
}'''

purlIris = set()

for k in sceneEntityASDict.keys():
    for v in sceneEntityASDict[k]:
        purlIris.add(v)
        

purlString = ", ".join("<" + s + ">" for s in purlIris)
queryResponse = None
try:
    queryResponse = api_instance.sparql(query=query % purlString, format='application/json')
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

In [10]:
def reverseObject(input_object):
    reversed_object = {}

    for key, values in input_object.items():
        for value in values:
            if value not in reversed_object:
                reversed_object[value] = set()
            reversed_object[value].add(key)
    return reversed_object

ASEntityDict = reverseObject(sceneEntityASDict)

In [77]:
sex = "both"
ontology_terms = ["http://purl.obolibrary.org/obo/UBERON_0002113",]
cell_type_terms = ["http://purl.obolibrary.org/obo/CL_0000000",]
tissueBlockResult = None
try:
    tissueBlockResult = api_instance.tissue_blocks(sex=sex, ontology_terms=ontology_terms, cell_type_terms=cell_type_terms)
except ccf_openapi_client.ApiException as e:
    print("Exception when calling DefaultApi->aggregate_results: %s\n" % e)

# Convert the TissueBlock Object to a dict
tissueBlockResultList = list(map(lambda b: b.to_dict(), tissueBlockResult))

In [78]:
blockDonorLabel = {}
for tissueBlock in tissueBlockResultList:
    blockDonorLabel[tissueBlock['@id']] = tissueBlock['donor']['label']

In [100]:
#Can't use set because - unhashable type: 'dict'
mergedData = []
for response in queryResponse:
    entity_ids = ASEntityDict[response['as_iri']]
    for e in entity_ids:
        newResponse = response.copy()
        newResponse['block_id'] = e
        newResponse['donor_label'] = blockDonorLabel[e]
        if newResponse not in mergedData:
            mergedData.append(newResponse)

In [102]:
import csv
def save_to_csv(data, headers, filename):
     with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)

        # Write the headers
        writer.writerow(headers)

        # Write the data
        for obj in data:
            row = [obj.get(headers[header], '') for header in headers]
            writer.writerow(row)

header_mapping = {
    'Block Id': 'block_id',
    'AS': 'as_iri',
    'AS Label': 'as_label',
    'CT': 'cell_iri',
    'CT Label': 'cell_label',
    'Donor Label':'donor_label'
}
save_to_csv(mergedData, header_mapping, 'output.csv')

In [103]:
df = pd.DataFrame(mergedData)
df.head(len(df))

,cell_label,as_label,cell_iri,as_iri,block_id,donor_label
0,"""T cell""","""kidney""",http://purl.obolibrary.org/obo/CL_0000084,http://purl.obolibrary.org/obo/UBERON_0002113,https://entity.api.hubmapconsortium.org/entiti...,"Female, Age 45, BMI 22.6"
1,"""T cell""","""kidney""",http://purl.obolibrary.org/obo/CL_0000084,http://purl.obolibrary.org/obo/UBERON_0002113,https://entity.api.hubmapconsortium.org/entiti...,"Male, Age 65"
2,"""T cell""","""kidney""",http://purl.obolibrary.org/obo/CL_0000084,http://purl.obolibrary.org/obo/UBERON_0002113,https://entity.api.hubmapconsortium.org/entiti...,"Female, Age 54"
3,"""T cell""","""kidney""",http://purl.obolibrary.org/obo/CL_0000084,http://purl.obolibrary.org/obo/UBERON_0002113,https://entity.api.hubmapconsortium.org/entiti...,"Male, Age 21, BMI 21.8"
4,"""T cell""","""kidney""",http://purl.obolibrary.org/obo/CL_0000084,http://purl.obolibrary.org/obo/UBERON_0002113,https://entity.api.hubmapconsortium.org/entiti...,"Male, Age 54, BMI 30.2"
...,...,...,...,...,...,...
6106,"""capsule mesenchymal stromal cell""","""kidney capsule""",https://purl.org/ccf/ASCTB-TEMP_capsule-mesenc...,http://purl.obolibrary.org/obo/UBERON_0002015,https://entity.api.hubmapconsortium.org/entiti...,"Male, Age 54, BMI 38.6"
6107,"""capsule mesenchymal stromal cell""","""kidney capsule""",https://purl.org/ccf/ASCTB-TEMP_capsule-mesenc...,http://purl.obolibrary.org/obo/UBERON_0002015,https://entity.api.hubmapconsortium.org/entiti...,"Female, Age 45, BMI 22.6"
6108,"""capsule mesenchymal stromal cell""","""kidney capsule""",https://purl.org/ccf/ASCTB-TEMP_capsule-mesenc...,http://purl.obolibrary.org/obo/UBERON_0002015,https://entity.api.hubmapconsortium.org/entiti...,"Female, Age 69, BMI 49.1"
6109,"""capsule mesenchymal stromal cell""","""kidney capsule""",https://purl.org/ccf/ASCTB-TEMP_capsule-mesenc...,http://purl.obolibrary.org/obo/UBERON_0002015,https://entity.api.hubmapconsortium.org/entiti...,"Male, Age 29, BMI 42.3"


In [104]:
# Summary -- texttt
AsCounts = df['as_iri'].value_counts()
CtCounts = df['cell_iri'].value_counts()
blockCounts = df['block_id'].value_counts()

print(f'SUMMARY:\n\
Number of Blocks: {len(blockCounts)},\n\
Number of Unique ASs identified: {len(AsCounts)},\n\
Number of Unique CTs identified: {len(CtCounts)}')

SUMMARY:
Number of Blocks: 83,
Number of Unique ASs identified: 8,
Number of Unique CTs identified: 68


In [ ]:
# Documentation
